# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )

In [6]:
#1. 10 countries with the lowest infant mortality rates
import pandas as pd
Dict = {'Name':[], 'Infant_mortality':[]}

#Retrive country names and infant mortality rates
for element in document.iterfind('country'):
    Dict['Name'] = Dict['Name'] + [element.find('name').text]
    if not element.find('infant_mortality') == None:
        Dict['Infant_mortality'] = Dict['Infant_mortality'] + [element.find('infant_mortality').text]
    else:
        Dict['Infant_mortality'] = Dict['Infant_mortality'] + ['NaN']

#Create dataframe and convert infant mortality to numbers        
Country_infant = pd.DataFrame(Dict)
Country_infant['Infant_mortality'] = pd.to_numeric(Country_infant['Infant_mortality'], errors='coerce')

#Sort infant mortality
result = Country_infant.sort_values(['Infant_mortality'])

#Print the 10 lowest
print('10 countries with the lowest infant mortality rates:')
print(result.iloc[0:10].to_string(index=False))

10 countries with the lowest infant mortality rates:
Infant_mortality            Name
            1.81          Monaco
            2.13           Japan
            2.48         Bermuda
            2.48          Norway
            2.53       Singapore
            2.60          Sweden
            2.63  Czech Republic
            2.73       Hong Kong
            3.13           Macao
            3.15         Iceland


In [7]:
#2. 10 cities with the largest population
root = document.getroot()
Dict2 = {'City':[], 'Population':[], 'Year':[]}

#find city names
for city in root.iter('city'):
    Dict2['City'] = Dict2['City'] + [city.find('name').text + '(' + city.attrib['country'] + ')']
    
    year,population = 0,0

    #find population and year 
    for pop in city.iter('population'):
        if int(pop.attrib['year']) > year:
            year = int(pop.attrib['year'])
            population = int(pop.text)
            
    Dict2['Population'] = Dict2['Population'] + [population]
    Dict2['Year'] = Dict2['Year'] + [year]

#Dictionary to dataframe
City_pop = pd.DataFrame(Dict2)

result = City_pop.sort_values(['Population'], ascending=False).iloc[0:10]
print('10 cities with the largest population (the most recent data available for each city)')
print(result)

10 cities with the largest population (the most recent data available for each city)
               City  Population  Year
1341   Shanghai(CN)    22315474  2010
771    Istanbul(TR)    13710512  2012
1527    Mumbai(IND)    12442373  2011
479       Moskva(R)    11979529  2013
1340    Beijing(CN)    11716620  2010
2810  São Paulo(BR)    11152344  2010
1342    Tianjin(CN)    11090314  2010
1064  Guangzhou(CN)    11071424  2010
1582     Delhi(IND)    11034555  2011
1067   Shenzhen(CN)    10358381  2010


In [8]:
#3. 10 ethnic groups with the largest overall populations 
# (sum of best/latest estimates over all countries)
root = document.getroot()
Dict3 = {}
Dict4 = {'Ethnicity':[], 'Population':[]}

#find the most recent population of a country
for country in root.findall("./country"):
    Country_pop = []
    for pop in country.findall('./population'):
        Country_pop = Country_pop + [pop.text]   
    Pop = int(Country_pop[-1])
    
    #find ethnic groups
    for ethnic in country.iter('ethnicgroup'):
        if not ethnic.text in Dict3:
            Dict3[ethnic.text] = int(Pop * float(ethnic.attrib['percentage']))
        else: 
            Dict3[ethnic.text] = int(Dict3[ethnic.text] + Pop * float(ethnic.attrib['percentage']))

Dict4['Ethnicity'] = list(Dict3.keys())
Dict4['Population'] = list(Dict3.values())
Ethnic_pop = pd.DataFrame(Dict4)
print('10 largest ethnic groups:')
print(Ethnic_pop.sort_values(['Population'], ascending=False).iloc[0:10].to_string(index=False))

10 largest ethnic groups:
Ethnicity    Population
Han Chinese  124505880000
 Indo-Aryan   87181558344
   European   49487221967
    African   31832512032
  Dravidian   30271374425
    Mestizo   15773435493
    Bengali   14677691672
    Russian   13185699603
   Japanese   12653421200
      Malay   12199355037


In [9]:
#Function to look up the country name for the country acronym 
def countrycode_to_countryname(data):
    CO = data['Country'].iloc[0].split()
    data['Country'] = ''
    for i in range(0, len(CO)):
        for j in root.findall('./country'):
            if CO[i] == j.attrib['car_code']:
                data['Country'] = data['Country'] + ' '+ j.find('name').text
                i+=1
                break
    return  data    

In [10]:
#4. name and country of a) longest river, b) largest lake and c) airport at highest elevation
root = document.getroot()

#find the longest river
Dict5={'Name':[], 'Length':[], 'Country':[]}
for river in root.findall('./river'):
    Dict5['Country'] = Dict5['Country'] + [river.attrib['country']]
    Dict5['Name'] = Dict5['Name'] + [river.find('name').text]
    if not river.find('length') == None:
        Dict5['Length'] = Dict5['Length'] + [float(river.find('length').text)]
    else:
        Dict5['Length'] = Dict5['Length'] + [0]
River_len = pd.DataFrame(Dict5)
result1 = River_len.sort_values(['Length'], ascending=False).iloc[0:1]
output = countrycode_to_countryname(result1)
print('The longest river:')
print(output)    

#find the largest lake
Dict6={'Name':[], 'Area':[], 'Country':[]}
for lake in root.findall('./lake'):
    Dict6['Country'] = Dict6['Country'] + [lake.attrib['country']]
    Dict6['Name'] = Dict6['Name'] + [lake.find('name').text]
    if not lake.find('area') == None:
        Dict6['Area'] = Dict6['Area'] + [float(lake.find('area').text)]
    else:
        Dict6['Area'] = Dict6['Area'] + [0]    
Lake_area = pd.DataFrame(Dict6)
result2 = Lake_area.sort_values(['Area'], ascending=False)[0:1]
output = countrycode_to_countryname(result2)
print('')
print('The largest lake:')
print(output)

#find the airport at highest elevation
Dict7={'Name':[], 'Elevation':[], 'Country':[]}
for air in root.iter('airport'):
    Dict7['Country'] = Dict7['Country'] + [air.attrib['country']]
    Dict7['Name'] = Dict7['Name'] + [air.find('name').text]
    Dict7['Elevation'] = Dict7['Elevation'] + [air.find('elevation').text]
Airport_elevation = pd.DataFrame(Dict7)
Airport_elevation['Elevation'] = pd.to_numeric(Airport_elevation['Elevation'], errors='coerce')
result3 = Airport_elevation.sort_values(['Elevation'], ascending=False)[0:1]
output = countrycode_to_countryname(result3)
print('')
print('The airport at highest elevation:')
print(output)

The longest river:
                   Country  Length      Name
174   Colombia Brazil Peru  6448.0  Amazonas

The largest lake:
        Area                                          Country         Name
54  386400.0   Russia Azerbaijan Kazakhstan Iran Turkmenistan  Caspian Sea

The airport at highest elevation:
     Country  Elevation          Name
80   Bolivia     4063.0  El Alto Intl
